# Neural network working as a decoder (decodes a binary sequence to the number)
## Example: [1,0,0,0,1,0,0,1] -> 137

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

## Create the dataset
- input: lists of size 8 with 0 and 1
- output: the value of the number

In [ ]:
samples = []
labels = []
for x in range(256):
    bin_val = bin(x)[2:].rjust(8,'0')
    samples.append(list(bin_val))
    labels.append(x)
samples = np.array(samples,dtype=float)
labels = np.array(labels,dtype=float)

for i in range(5):
    print(samples[i],' -> ',labels[i])    
print('...')
for i in range(250,256):
    print(samples[i],' -> ',labels[i])

## Build the neural network model

In [ ]:
ADDITIONAL_LAYER = True

model = Sequential()
model.add(Dense(12,input_dim=8))
if ADDITIONAL_LAYER:
    model.add(Dense(12))
model.add(Dense(1))
model.summary()

## Compile the model

In [ ]:
model.compile(loss='mean_absolute_error', optimizer="adam",metrics=['mae'])

## Divide to training and test 

In [ ]:
TRAIN_SIZE = 0.1 # only 10% to train!

import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples, labels, 
                            train_size=TRAIN_SIZE)
print('Training samples:',len(trainSamples),' test samples:',len(testSamples))

## Train the network

In [ ]:
EPOCHS = 200
print("Training using ",len(trainSamples),'samples and ',EPOCHS,' epochs')

H = model.fit(trainSamples,trainLabels,validation_data=(testSamples,testLabels), epochs=EPOCHS, verbose=1)
print("Initial loss - training mae={:6.3f} validation mae={:6.3f}".format(H.history['mae'][0],H.history['val_mae'][0]))
print("Final loss - training mae={:6.3f} validation mae={:6.3f}".format(H.history['mae'][-1],H.history['val_mae'][-1]))

## Calculate errors

In [ ]:
def calc_errors(predictedLabels,realLabels,verbose=1):
    predictedLabels = np.ravel(predictedLabels)
    for i in range(len(realLabels)):
        tNum = realLabels[i]
        pNum = predictedLabels[i]
        if(verbose>0):
            print("{:3.0f} -> {:6.2f} error={:7.2f}".format(tNum,pNum,abs(pNum-tNum)))
    print('Mean Absolute Error (MAE)= {:7.2f}'.format(np.mean(np.absolute(predictedLabels - realLabels))))
print('TRAINING SET')
predictedLabels = model.predict(trainSamples)
calc_errors(predictedLabels,trainLabels)
print()

print('TEST SET')
predictedLabels = model.predict(testSamples)
calc_errors(predictedLabels,testLabels)
